In [1]:
# 均方误差、小批量随机梯度下降

import torch
from time import time

if torch.cuda.is_available():
    device = torch.device("cuda")

#测试两种加法耗时
a = torch.randn(1000, device = device)
b = torch.randn(1000, device = device)
print(a.device, a.requires_grad)

cuda:0 False


In [3]:
start = time()
c = torch.zeros(1000)
for i in range(1000):
    c[i] = a[i] + b[i]
print(time() - start, c.device)

0.0651559829711914 cpu


In [5]:
#更快
start = time()
c = a + b
print(time() - start, c.device)
#直接定义的c是在cpu上，由gpu上的tensor运算得到的c是在gpu上

0.0 cuda:0


In [7]:
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [ ]:
num_inputs = 2 #特征数
num_examples = 1000 #样本数

#在实际问题上true_w喝true_b不知道，只不过这里是在模拟数据，是用来计算真实的y的
true_w = [2, -3.4]
true_b = 4.2

"""
#np.random.normal(loc, scale, size)中loc是期望，scale是标准差，size是大小
features = torch.from_numpy(np.random.normal(0, 1, (num_examples, num_inputs)))
#要dtype = float，features和int的w不能乘
labels = features @ torch.tensor(true_w, dtype = float).t() + true_b
labels += torch.from_numpy(np.random.normal(0, 0.01, labels.size()))
#print(labels)
"""

"""
以上代码会导致kernel内核挂掉，疑似为以下原因
1.float类型冲突，所以全指定torch.float32
2..t()转置使用错误，true_w = [2, -3.4]是一个一维向量，size为[2]，而.t() 只对二维张量转置
    所以可以直接不用转置，因为pytorch的特性，让 1D 张量既可以当行向量，也可以当列向量用
3.torch.from_numpy() 和 np.random.normal() 生成的数组是共享内存的，可能导致崩溃，所以用torch.tensor()不共享内存
"""
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype = torch.float32)
labels = features @ torch.tensor(true_w, dtype = torch.float32)+ true_b
labels += torch.tensor(np.random.normal(0, 0.01, size = labels.size()), dtype = torch.float32)


#好吧还是崩溃了。。重开一个book吧


fig = plt.figure(figsize = (20, 8), dpi = 80)
plt.scatter(features[:, 0].numpy(),labels.numpy())
plt.show()